# Chatbot with Memory

Let's add some history to the interaction and build a chatbot. Unlike many people think. LLMs are fixed in their state. They are trained until a certain cutoff date and do not know anything after that point unless you feed them current information. That is also why LLMs do not remember anything about you or the prompts you send to the model. If the model seems to remember you and what you said it is always because the application you are using (e.g. ChatPGT or the chat function in SAP AI Launchpad) is sending the chat history to the model to provide the conversation history to the model as context.

Below you can find a simple implementation of a chatbot with memory.

The code in this exercise is based on the [help documentation](https://help.sap.com/doc/generative-ai-hub-sdk/CLOUD/en-US/_reference/orchestration-service.html) of the Generative AI Hub Python SDK.

In [1]:
import init_env
import variables

init_env.set_environment_variables()

In [2]:
from gen_ai_hub.orchestration.models.llm import LLM
from gen_ai_hub.orchestration.models.message import Message, SystemMessage, UserMessage
from gen_ai_hub.orchestration.models.template import Template, TemplateValue
from gen_ai_hub.orchestration.models.config import OrchestrationConfig
from gen_ai_hub.orchestration.service import OrchestrationService

In [3]:
class ChatBot:
    def __init__(self, orchestration_service: OrchestrationService):
        self.service = orchestration_service
        self.config = OrchestrationConfig(
            template=Template(
                messages=[
                    SystemMessage("You are a helpful chatbot assistant."),
                    UserMessage("{{?user_query}}"),
                ],
            ),
            llm=LLM(name="gemini-1.5-flash"),
        )
        self.history: List[Message] = []

    def chat(self, user_input):
        response = self.service.run(
            config=self.config,
            template_values=[
                TemplateValue(name="user_query", value=user_input),
            ],
            history=self.history,
        )

        message = response.orchestration_result.choices[0].message

        self.history = response.module_results.templating
        self.history.append(message)

        return message.content
    
    def reset(self):
        self.history = []

service = OrchestrationService(api_url=variables.AICORE_ORCHESTRATION_DEPLOYMENT_URL)
bot = ChatBot(orchestration_service=service)

In [4]:
bot.chat("Hello, how are you?")

"Hello! I'm doing well, thank you for asking.  How can I help you today?\n"

In [5]:
print(bot.chat("What's the weather like today?"))
bot.history

I do not have access to real-time information, including weather. To get the weather for your location, please check a reliable weather app or website such as Google Weather, AccuWeather, or your local news.



[Message(role=<Role.SYSTEM: 'system'>, content='You are a helpful chatbot assistant.'),
 Message(role=<Role.USER: 'user'>, content='Hello, how are you?'),
 Message(role=<Role.ASSISTANT: 'assistant'>, content="Hello! I'm doing well, thank you for asking.  How can I help you today?\n"),
 Message(role=<Role.SYSTEM: 'system'>, content='You are a helpful chatbot assistant.'),
 Message(role=<Role.USER: 'user'>, content="What's the weather like today?"),
 Message(role=<Role.ASSISTANT: 'assistant'>, content='I do not have access to real-time information, including weather. To get the weather for your location, please check a reliable weather app or website such as Google Weather, AccuWeather, or your local news.\n')]

In [6]:
print(bot.chat("Can you remember what I first asked you?"))

Yes, your first question was "Hello, how are you?"



And to prove to you that the model does indeed not remember you, let's delete the history and try again :)

In [7]:
bot.reset()
print(bot.chat("Can you remember what I first asked you?"))

As a large language model, I have no memory of past conversations.  Each interaction with me starts fresh.  To help me assist you, please tell me what you'd like me to do now.



[Next exercise - OPTIONAL](11-ai-agents.ipynb)